In [4]:
# Import library yang diperlukan
import pandas as pd
import numpy as np
from gensim import corpora, models
import gensim
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import ast

# Baca dataset
df = pd.read_csv('../../dataset/cleaned-data/CyberU_reviews_cleaned.csv')

# Konversi string representasi list menjadi list yang sebenarnya
df['tokenized_text'] = df['tokenized_text'].apply(ast.literal_eval)

# Buat Dictionary
dictionary = corpora.Dictionary(df['tokenized_text'])

# Buat Corpus
corpus = [dictionary.doc2bow(text) for text in df['tokenized_text']]

# Fungsi untuk menghitung coherence score
def compute_coherence_values(dictionary, corpus, texts, start, limit, step):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               num_topics=num_topics, 
                                               id2word=dictionary,
                                               random_state=42,
                                               passes=15)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, 
                                      dictionary=dictionary, 
                                      coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

# Hitung coherence score untuk berbagai jumlah topik
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, 
                                                      corpus=corpus,
                                                      texts=df['tokenized_text'],
                                                      start=2, limit=11, step=1)



^C
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'gensim'

  error: subprocess-exited-with-error
  
  × Building wheel for gensim (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [102 lines of output]
      C:\Users\hp\AppData\Local\Temp\pip-build-env-pl0efaun\overlay\Lib\site-packages\setuptools\_distutils\dist.py:270: UserWarning: Unknown distribution option: 'test_suite'
        warnings.warn(msg)
      C:\Users\hp\AppData\Local\Temp\pip-build-env-pl0efaun\overlay\Lib\site-packages\setuptools\_distutils\dist.py:270: UserWarning: Unknown distribution option: 'tests_require'
        warnings.warn(msg)
      running bdist_wheel
      running build
      running build_ext
      Traceback (most recent call last):
        File "c:\Users\hp\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
          ~~~~^^
        File "c:\Users\hp\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\

  Using cached gensim-4.3.3.tar.gz (23.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached numpy-1.26.4.tar.gz (15.8 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): still running...
  Preparing metadata (pyproject.toml): still running...
  Preparing metadata (pyproject.toml): still running...
  Preparing metadata (pyproject.toml): stil

In [ ]:
# Plot coherence scores
plt.plot(range(2, 11), coherence_values)
plt.xlabel("Jumlah Topik")
plt.ylabel("Coherence Score")
plt.show()

In [ ]:
# Pilih model dengan coherence score terbaik
optimal_model = model_list[np.argmax(coherence_values)]

# Visualisasi dengan pyLDAvis
vis = pyLDAvis.gensim_models.prepare(optimal_model, corpus, dictionary)
pyLDAvis.save_html(vis, 'lda_visualization.html')

# Buat wordcloud untuk setiap topik
def plot_word_cloud(model, topic_number):
    topic_words = dict(model.show_topic(topic_number, 30))
    wordcloud = WordCloud(width=800, height=400, 
                         background_color='white').generate_from_frequencies(topic_words)
    
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Topik {topic_number}')
    plt.show()

In [ ]:
# Plot wordcloud untuk setiap topik
for i in range(optimal_model.num_topics):
    plot_word_cloud(optimal_model, i)